# Analysis of Quora Insincere Questions Classification

This is going to be a project desined by Alejandro Guarnizod de la Torre, where i will be using and applying all the knoledge that i have adquire and expand on it for the purporse of learning more about NLP techniques. 

## Data processing
In this first part i will be cleaning the data, and checking that we are not missing anything, so that it is correctly prepared to feed into an embedding and later on into a model for testing. 

### Creating the enviroment

In [1]:
import pandas as pd
import re

In [2]:
df_train = pd.read_csv("../data/raw/train.csv")

In [3]:
df_train.head(10)

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
5,00004f9a462a357c33be,"Is Gaza slowly becoming Auschwitz, Dachau or T...",0
6,00005059a06ee19e11ad,Why does Quora automatically ban conservative ...,0
7,0000559f875832745e2e,Is it crazy if I wash or wipe my groceries off...,0
8,00005bd3426b2d0c8305,"Is there such a thing as dressing moderately, ...",0
9,00006e6928c5df60eacb,Is it just me or have you ever been in this ph...,0


In [4]:
df_train.columns

Index(['qid', 'question_text', 'target'], dtype='object')

I am not sure how to increase the window size in VS so i just cheked the data in the csv file, in any case, it does not seem to have any weird characters or anything, but just in case i will run a few funtions that are commonly used when cleaning data, to remove unwanted characters. So lets do that next and then continue preparing the data for training and testing. 

We will remove weird characters, which usually embeddings, (techniques to make the characters into vector that machine learning can procress) since usually they are not trained with it. 
I do not think there will be emojies in the text, so we did not incluse those in the list of symbols to remove. Although it would be more easier to remove with a regex operation, that filters anything that is not a character. The only issue is that we do not have a direct list where we can check everything that will be removed. 

In [5]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', ' ', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, '')
    return x

In [ ]:
# alternative regex expression. 
def clean_text(x):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', x)
    return x

In [6]:
df_train['question_text'] = df_train['question_text'].apply(clean_text)

It is also common to remove numbers, since most embeddings do not have them trained with it, so in this case following the guideliness i will also remove them. 

In [7]:
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [8]:
df_train['question_text'] = df_train['question_text'].apply(clean_numbers)

### Removing misspells. 
Now something that we must do, to improve accuracy is to remove misspell words, i find it really amazing that there have models created for this purpose. According to the research the following code artifact is an adaptation of Peter Norvig’s spell checker. It uses word2vec ordering of words to approximate word probabilities. Word2vec, a model created from google, it orders the words in terms of frequency, we can then suppose that the misspelled words will have really low frequence since they are involuntary mistakes, that is why they will be few. If that is the case, then we can set up a cut of frequency, and stablish whish ones are misspelled. 

In [9]:
import re
from collections import Counter
import gensim
import heapq
from operator import itemgetter
from multiprocessing import Pool

# we load the model
model = gensim.models.KeyedVectors.load_word2vec_format('../models/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin', binary=True)

words = model.index_to_key # this is something related to the new version, it has changed. 

# this is to create a ranking, basically count the number of words. We are using a hashtable, because it is more easy to count things. 
w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i

WORDS = w_rank

def words(text): 
    # this function makes every world lowercase, that way it is consistent with everything and there will be no problems. 
    return re.findall(r'\w+', text.lower())

def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

vocab = build_vocab(df_train.question_text)

# we use a heapq to order the ranking so that we can get fast the most popular. 
top_90k_words = dict(heapq.nlargest(90000, vocab.items(), key=itemgetter(1)))

# we make a pool of Process so we can divide the tasks and finish more faster. 
pool = Pool(4)
corrected_words = pool.map(correction,list(top_90k_words.keys()))

for word,corrected_word in zip(top_90k_words,corrected_words):
    if word!=corrected_word:
        print(word,":",corrected_word)